# As per https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
I will test both a simple RNN and CNN

https://github.com/streamride/wakeworddetection/blob/main/model.py

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleRNN(nn.Module):
    def __init__(self, spec_time, dropout_rate=0.5):
        super().__init__()
        hidden_layer = int(2*spec_time)
        hidden_layer2 = int(spec_time/2)
        
        self.rnn = nn.LSTM(spec_time,hidden_layer, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_layer, spec_time),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(spec_time, hidden_layer2),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer2,1)
        )
        # self.fc1 = nn.Linear(256, 128)
        # self.fc = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_rate)

        self.layer_norm = nn.LayerNorm(hidden_layer2)

    # def init_hidden(self, batch_size):
        # return 

    def forward(self, x):
        x = x.squeeze(1)
        x = self.layer_norm(x)
        # print(x.shape)
        # x = x.transpose(0,1)
        # x = x.transpose(1,2)
        
        # print(x.shape)
        x, (hidden_last, cell_last) = self.rnn(x)
        hidden_last = self.dropout(hidden_last)
        # print(x.shape)
        # print(hidden_last.shape)
        hidden_last = hidden_last.squeeze(0)
        x = self.classifier(hidden_last)
        # print(x.shape)
        return x

In [29]:
from torchsummary import summary
input_time = 1375

model = SimpleRNN(spec_time = 128)

print(f"input shape: (1,{101},{input_time})")  
summary(model, (128,64))    

input shape: (1,101,1375)


RuntimeError: input.size(-1) must be equal to input_size. Expected 128, got 64

In [20]:
spec_time = 128
hidden_layer = int(2*spec_time)
hidden_layer2 = int(spec_time/2)
dropout_rate = 0.8

rnn = nn.LSTM(spec_time,hidden_layer, batch_first=True)
classifier = nn.Sequential(
nn.Linear(hidden_layer, spec_time),
nn.LeakyReLU(),
nn.Dropout(dropout_rate),
nn.Linear(spec_time, hidden_layer2),
nn.LeakyReLU(),
nn.Linear(hidden_layer2,1)
)
# self.fc1 = nn.Linear(256, 128)
# self.fc = nn.Linear(128, 1)
dropout = nn.Dropout(dropout_rate)

layer_norm = nn.LayerNorm(hidden_layer2)